In [1]:
from Crypto.PublicKey import RSA as rsa
from Crypto.Signature import PKCS1_v1_5 as sig
from Crypto.Hash import SHA
from Crypto.Cipher import XOR
import os
import gmpy2

In [18]:
Generate_New_Signer_Key = 1
message = b'Test this message'
#This includes the real signer.  So we remove one when generating keys later.
total_ring_size = 5
key_size = 1024

In [10]:
#Generate a new key for the real signer
if Generate_New_Signer_Key == 1:
    signer_key = rsa.generate(key_size, os.urandom)
    f = open('mykey.der','wb')
    f.write(signer_key.exportKey('DER'))
    f.close()

In [4]:
def ring_public_keys(signer_count, key_size):
    b = 0
    circle_pub_keys = []
    circle_n = []
    while b < (signer_count - 1):
        circle_key = rsa.generate(key_size, os.urandom)
        public_key = circle_key.publickey().exportKey(format='DER')
        circle_pub_keys.append(public_key)
        circle_n.append(circle_key.n)
        b += 1
    return circle_pub_keys, circle_n
    
    

In [6]:
public_key_set, public_n_set = ring_public_keys(total_ring_size, key_size)

In [7]:
#This will sign the outputs from the original signer.  This does not execute the trap door function.
def sign_by_ring(public_key_set, public_n_set, v):
    for i in public_key_set:
        for j in public_n_set:
            s = os.urandom
            e = gmpy2.powmod(s,int.from_bytes(i,'big'),j)
            cipher = XOR.new(e)
            v = cipher.encrypt(v)
    return v
    

In [15]:
#Create the first signature from the real signer
g = open('mykey.der','rb')
key = rsa.importKey(g.read())
h = SHA.new(message)
signer = sig.new(key)
signature = signer.sign(h)

In [21]:
#I need to spend a little time thinking about what the real signer sends to the second person in the ring
cipher = XOR.new(signature)

ValueError: XOR key must be no longer than 32 bytes

In [36]:
hashed = SHA.new(message)

In [6]:
signer = sig.new(key)

In [7]:
signature = signer.sign(hashed)

### Verify a signature

In [59]:
h = SHA.new(message)
verifier = sig.new(key)
if verifier.verify(h, signature):
    print("The signature is authentic.")
else:
    print("The signature is not authentic.")

The signature is authentic.
